In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv('storepurchasedata_large.csv')

In [4]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [5]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [6]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [10]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [11]:
#Converting to tensor format
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [12]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [13]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [14]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [15]:
#input size for features, output for labels, hidden equals number of neurons in layers
input_size=2
output_size=2
hidden_size=10

In [16]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size) #fc = fully connected layer. All neurons connected, Input to hidden
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size) #hidden to hidden layer
       self.fc3 = torch.nn.Linear(hidden_size, output_size) #hiddent to output. two hidden layers total


   def forward(self, X):
       X = torch.relu((self.fc1(X))) #relu activation in hyden layers. Helps find when values become useful. EX, people do not start buying prodcuts until age 20 
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1) #softmax converts neural output to a multinomial probability distribution for classification problems. 

In [17]:
model = Net()



In [18]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [19]:
epochs = 100


In [20]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_) #tracking loss on each epoch.
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.7282726764678955
Epoch 1 loss 0.7126734256744385
Epoch 2 loss 0.6962888240814209
Epoch 3 loss 0.6796692609786987
Epoch 4 loss 0.66259765625
Epoch 5 loss 0.6451107263565063
Epoch 6 loss 0.6267644166946411
Epoch 7 loss 0.607507050037384
Epoch 8 loss 0.5873623490333557
Epoch 9 loss 0.5662058591842651
Epoch 10 loss 0.5443200469017029
Epoch 11 loss 0.5218295454978943
Epoch 12 loss 0.4986516237258911
Epoch 13 loss 0.47522640228271484
Epoch 14 loss 0.4517792761325836
Epoch 15 loss 0.4289405047893524
Epoch 16 loss 0.4070834815502167
Epoch 17 loss 0.38679584860801697
Epoch 18 loss 0.3682653307914734
Epoch 19 loss 0.3508813679218292
Epoch 20 loss 0.33476781845092773
Epoch 21 loss 0.3198454678058624
Epoch 22 loss 0.3061002492904663
Epoch 23 loss 0.2931402325630188
Epoch 24 loss 0.2806519865989685
Epoch 25 loss 0.26860377192497253
Epoch 26 loss 0.2571456730365753
Epoch 27 loss 0.24620980024337769
Epoch 28 loss 0.23596009612083435
Epoch 29 loss 0.22626672685146332
Epoch 30 loss 0.217

In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3622, -1.0820],
         [-0.1485, -0.7900],
         [-0.1059, -0.2297],
         [ 0.4338, -1.2020],
         [ 0.9556,  0.1003],
         [-0.0489,  0.0922],
         [-0.3906, -0.6401],
         [ 0.8760,  0.9435],
         [ 1.1214, -1.0266],
         [ 1.1170,  0.6277]], requires_grad=True), Parameter containing:
 tensor([ 0.0582,  0.1692,  0.6380,  0.0734,  0.2235, -0.3857,  0.2636,  0.4070,
         -0.4218,  0.5164], requires_grad=True), Parameter containing:
 tensor([[-0.4883, -0.2586,  0.2851, -0.2845,  0.5507, -0.2682,  0.1875,  0.5617,
           0.5647,  0.2195],
         [-0.0209,  0.0436,  0.2692, -0.1608, -0.2071,  0.1825, -0.2062,  0.2406,
          -0.1111, -0.2608],
         [-0.4570, -0.2578,  0.0780,  0.5636,  0.5373, -0.0652, -0.4591,  0.1822,
           0.7904,  0.3245],
         [-0.2859, -0.1058, -0.2531, -0.6481,  0.7085,  0.0335,  0.3802,  0.2904,
           0.9382,  0.3937],
         [-0.2324, -0.2757, -0.0809,  0.2176,  

In [22]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000 #first output would be 0 (not buying), or 1 (buying) make sure data is in Torch format and scaled. 

tensor([[-1.5695e-03, -6.4578e+00]], grad_fn=<LogSoftmaxBackward0>)

In [23]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000 #using max, gives you the index for the highest probability

tensor([0])

In [24]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-0.8554, -0.5536]], grad_fn=<LogSoftmaxBackward0>)

In [25]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

In [26]:
#saving for later
torch.save(model,'customer_neuralnetwork.pt')

In [27]:
!ls

customer_neuralnetwork.pt  sample_data


In [28]:
restored_model = torch.load('customer_neuralnetwork.pt')

In [29]:
y_cust_20_40000 = restored_model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-1.5695e-03, -6.4578e+00]], grad_fn=<LogSoftmaxBackward0>)

In [30]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [31]:
model.state_dict()

OrderedDict([('fc1.weight', tensor([[-0.3622, -1.0820],
                      [-0.1485, -0.7900],
                      [-0.1059, -0.2297],
                      [ 0.4338, -1.2020],
                      [ 0.9556,  0.1003],
                      [-0.0489,  0.0922],
                      [-0.3906, -0.6401],
                      [ 0.8760,  0.9435],
                      [ 1.1214, -1.0266],
                      [ 1.1170,  0.6277]])),
             ('fc1.bias',
              tensor([ 0.0582,  0.1692,  0.6380,  0.0734,  0.2235, -0.3857,  0.2636,  0.4070,
                      -0.4218,  0.5164])),
             ('fc2.weight',
              tensor([[-0.4883, -0.2586,  0.2851, -0.2845,  0.5507, -0.2682,  0.1875,  0.5617,
                        0.5647,  0.2195],
                      [-0.0209,  0.0436,  0.2692, -0.1608, -0.2071,  0.1825, -0.2062,  0.2406,
                       -0.1111, -0.2608],
                      [-0.4570, -0.2578,  0.0780,  0.5636,  0.5373, -0.0652, -0.4591,  0.1822,
   

In [32]:
#saving using dictionary
torch.save(model.state_dict(),'customer_dict_neuralnetwork')

In [33]:
!ls

customer_dict_neuralnetwork  customer_neuralnetwork.pt	sample_data


In [34]:
new_predictor = Net()

In [35]:
y_cust_20_40000 = new_predictor(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.7785, -0.6145]], grad_fn=<LogSoftmaxBackward0>)

In [36]:
!zip -r customer_dict_neuralnetwork.zip customer_dict_neuralnetwork

  adding: customer_dict_neuralnetwork (deflated 47%)


In [37]:
!ls

customer_dict_neuralnetwork	 customer_neuralnetwork.pt
customer_dict_neuralnetwork.zip  sample_data


In [38]:
from google.colab import files


In [39]:
#files.download('customer_dict_neuralnetwork.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>